# Notebook to retrieve gridded climate time-series data sets

### Case study: the Sauk-Suiattle river watershed, the Elwha river watershed, the Upper Rio Salado watershed
<img src="http://www.sauk-suiattle.com/images/Elliott.jpg" 
style="float:right;width:150px;padding:20px">

### Use this Jupyter Notebook to:
    1. HydroShare setup and preparation
    2. Re-establish the paths to the mapping file
    3. Download climate data
    4. Summarize the file availability from each watershed mapping file
    5. Save results back into HydroShare

<br/><br/><br/>
<img src="https://www.washington.edu/brand/files/2014/09/W-Logo_Purple_Hex.png"
style="float:right;width:150px;padding:20px">

<br/><br/>
#### This data is compiled to digitally observe the watersheds, powered by HydroShare. <br/>Provided by the Watershed Dynamics Group, Dept. of Civil and Environmental Engineering, University of Washington

## 1.  HydroShare Setup and Preparation

To run this notebook, we must import several libaries. These are listed in order of 1) Python standard libraries, 2) hs_utils library provides functions for interacting with HydroShare, including resource querying, dowloading and creation, and 3) the observatory_gridded_hydromet library that is downloaded with this notebook. 

In [ ]:
# data processing
import os
import pandas as pd, numpy as np, dask, json
import ogh
import geopandas as gpd
import ogh_xarray_landlab as oxl
import xarray as xray

# data migration library
from utilities import hydroshare

# plotting and shape libraries
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

Establish a secure connection with HydroShare by instantiating the hydroshare class that is defined within hs_utils. In addition to connecting with HydroShare, this command also sets and prints environment variables for several parameters that will be useful for saving work back to HydroShare. 

In [ ]:
notebookdir = os.getcwd()

hs=hydroshare.hydroshare()
homedir = hs.getContentPath(os.environ["HS_RES_ID"])
os.chdir(homedir)
print('Data will be loaded from and save to:'+homedir)

In [ ]:
# initialize ogh_meta
meta_file = dict(ogh.ogh_meta())
sorted(meta_file.keys())

In [ ]:
"""
1/16-degree Gridded cell centroids
"""
# List of available data
hs.getResourceFromHydroShare('ef2d82bf960144b4bfb1bae6242bcc7f')
NAmer = hs.content['NAmer_dem_list.shp']


"""
Sauk
"""
# Watershed extent
hs.getResourceFromHydroShare('c532e0578e974201a0bc40a37ef2d284')
sauk = hs.content['wbdhub12_17110006_WGS84_Basin.shp']

# reproject the shapefile into WGS84
ogh.reprojShapefile(sourcepath=sauk)

In [ ]:
import wget


# retrieve the netcdf map
domain='http://cses.washington.edu'
subdomain='rocinante/WRF/PNNL_NARR_6km'
netcdfmap = 'data_LatLonGht.nc'

if not os.path.exists(netcdfmap):
    wget.download('{0}/{1}/{2}'.format(domain, subdomain, netcdfmap))

pnnlxy=xray.open_dataset(netcdfmap)
pnnlxy

In [ ]:
pnnlxy_df = pnnlxy.to_dataframe().reset_index().rename(columns={'south_north':'SN','west_east':'WE','Z':'ELEV'})
pnnlxy_df.tail()

# 2. Re-establish the paths to the mapping file

In [ ]:
# map the mapping files generated for Sauk-Suiattle, Elwha, and Upper Rio Salado from usecase1
mappingfile1 = os.path.join(homedir,'Sauk_mappingfile_i.csv')

t1 = ogh.mappingfileSummary(listofmappingfiles = [mappingfile1], 
                            listofwatershednames = ['Sauk-Suiattle river'],
                            meta_file=meta_file)

t1

In [ ]:
homedir

In [ ]:
mappingfile1

## 3. Download climate data 

### Call each get Climate data function

Each function reads in the mapping file table, generates the destination folder, downloads and unzips the files, then catalogs the downloaded files within the mapping file.

Meteorology - MET; Weather Research and Forecasting (WRF); Variable Infiltration Capacity - VIC

    1. getDailyMET_livneh2013
    2. getDailyMET_bcLivneh2013
    3. getDailyMET_livneh2015
    4. getDailyVIC_livneh2013
    5. getDailyVIC_livneh2015
    6. getDailyWRF_salathe2014
    7. getDailyWRF_bcsalathe2014

### View data extent:

    1. Continental United States (CONUS)
    Livneh, B. (2017). Gridded climatology locations (1/16th degree): Continental United States extent, HydroShare, http://www.hydroshare.org/resource/14f0a6619c6b45cc90d1f8cabc4129af

    2. North America (NAmer)
    Livneh, B. (2017). Gridded climatology locations (1/16th degree): North American extent, HydroShare, http://www.hydroshare.org/resource/ef2d82bf960144b4bfb1bae6242bcc7f

    3. Pacific Northwest - Columbia River Basin
    Bandaragoda, C. (2017). Sauk Suiattle HydroMeteorology (WRF), HydroShare, http://www.hydroshare.org/resource/0db969e4cfb54cb18b4e1a2014a26c82



### Please cite:

    1. Livneh B., E.A. Rosenberg, C. Lin, B. Nijssen, V. Mishra, K.M. Andreadis, E.P. Maurer, and D.P. Lettenmaier, 2013: A Long-Term Hydrologically Based Dataset of Land Surface Fluxes and States for the Conterminous United States: Update and Extensions, Journal of Climate, 26, 9384–9392.

    2. Livneh B., T.J. Bohn, D.S. Pierce, F. Munoz-Ariola, B. Nijssen, R. Vose, D. Cayan, and L.D. Brekke, 2015: A spatially comprehensive, hydrometeorological data set for Mexico, the U.S., and southern Canada 1950-2013, Nature Scientific Data, 5:150042, doi:10.1038/sdata.2015.42.

    3. Salathé, EP, AF Hamlet, CF Mass, M Stumbaugh, S-Y Lee, R Steed: 2017. Estimates of 21st Century Flood Risk in the Pacific Northwest Based on Regional Scale Climate Model Simulations.  J. Hydrometeorology. DOI: 10.1175/JHM-D-13-0137.1

### Download all available datasets for Sauk-Suiattle Watershed

In [ ]:
# map_df = pd.read_csv(mappingfile1)
# map_df
# len(map_df)
# map_df.dropna()

# ds_pan.loc['48.12695']

In [ ]:
maptable, nstations = ogh.mappingfileToDF_PNNL(mappingfile1)
maptable

In [ ]:
spatialbounds = {'minx':maptable.WE.min().astype(np.int64), 'maxx':maptable.WE.max().astype(np.int64),
                 'miny':maptable.SN.min().astype(np.int64), 'maxy':maptable.SN.max().astype(np.int64)}
spatialbounds

In [ ]:
%%time
outputfiles2 = oxl.get_x_hourlywrf_PNNL2018(homedir=homedir,
                                            spatialbounds=spatialbounds,
                                            subdir='PNNL2018/Hourly_WRF_2005_2007/noBC',
                                            nworkers=40,
                                            start_date='2005-10-01',
                                            end_date='2007-09-30',
                                            file_prefix='sp_',
                                            replace_file=True)

In [ ]:
outputfiles2

## Troubleshoot get_x_hourlywrf_PNNL2018

In [ ]:
#troublshoot 
# generate the list of files to download
        # modify each month between start_date and end_date to year-month
start_date='2005-10-01'
end_date='2007-09-30'
dates = [x.strftime('%Y%m%d') for x in pd.date_range(start=start_date, end=end_date, freq='D')]
filelist = oxl.compile_x_wrfpnnl2018_raw_locations(dates)
fileurl = filelist[1]
basename = os.path.basename(fileurl)


In [ ]:
!ls -l PNNL2018/Hourly_WRF_2005_2007/noBC

In [ ]:
test = xr.open_dataset('PNNL2018/Hourly_WRF_2005_2007/noBC/'+'sp_'+os.path.basename(basename))
test2 = xr.open_mfdataset(outputfiles2)

In [ ]:
test2

In [ ]:
# this should be included into the variable_info meta_file section
#dict(test2.variables)

In [ ]:
ds_vars

In [ ]:
# generate list of variables
ds_vars = [ds_var for ds_var in dict(test2.variables).keys() 
           if ds_var not in ['YEAR','MONTH','DAY','TIME','LAT','LON', 'SN','WE']]

# convert netcdfs to pandas.Panel API
ds_pan = test2.to_dataframe()#[ds_vars]
ds_pan

In [ ]:
%%time

for ind, eachrow in maptable.iterrows():
    
    dummy = test2.sel(SN=eachrow['SN'], WE=eachrow['WE']).to_dataframe()
    
    print(dummy)
    
    if ind==5:
        break



In [ ]:
# def wget_x_download_spSubset_PNNL(fileurl,
#                                   filedate,
#                                   spatialbounds,
#                                   time_resolution='H',
#                                   time_steps=24,
#                                   file_prefix='sp_',
#                                   rename_timelatlong_names={'south_north':'SN','west_east':'WE'},
#                                   replace_file=True):
#         """
#         Download files from an http domain

#         fileurl: (str) a urls to request a netcdf file
#         spatialbounds: (dict) dict providing the minx, miny, maxx, and maxy of the spatial region
#         file_prefix: (str) a string to mark the output file as a spatial subset
#         rename_latlong_names: (dict) a dict to standardize latitude/longitude synonyms to LAT/LON, respectively
#         replace_file: (logic) If True, the existing file will be replaced; if False, the file download is skipped
#         """

#         # check if the file path already exists; if so, apply replace_file logic
#         basename = os.path.basename(fileurl)
#         if os.path.isfile(basename):
#             os.remove(basename)

#         if os.path.isfile(file_prefix+basename) and replace_file:
#             os.remove(file_prefix+basename)
#         elif os.path.isfile(file_prefix+basename) and not replace_file:
#             # replace_file is False; return file path and skip
#             return(os.path.join(os.getcwd(), file_prefix+basename))

#         # try the file connection
#         #print('connecting to: '+basename)
#         try:
#             ping = urllib.request.urlopen(fileurl)

#             # if the file exists, download it
#             if ping.getcode() != 404:
#                 ping.close()
#                 wget.download(fileurl)

#                 # open the parent netcdf file
#                 ds = xray.open_dataset(basename, engine = 'netcdf4')
#                 #print('file read in')
#                 # rename latlong if they are not LAT and LON, respectively
#                 if not isinstance(rename_timelatlong_names, type(None)):
#                     ds = ds.rename(rename_timelatlong_names)
#                     #print('renamed columns')

#                 # slice by the bounding box NOTE:dataframe slice includes last index
#                 ds=ds.assign_coords(SN=ds.SN, WE=ds.WE)
#                 spSubset = ds.sel(WE=slice(spatialbounds['minx'], spatialbounds['maxx']),
#                                   SN=slice(spatialbounds['miny'], spatialbounds['maxy']))
#                 print('cropped')
                
#                 # change time to datetimeindex
#                 hour = [x.strftime('%Y-%m-%d %H:%M:%S') for x in pd.date_range(start=filedate,
#                                                                                periods=time_steps,
#                                                                                freq=time_resolution)]
#                 spSubset['TIME']=pd.DatetimeIndex(hour)
                
#                 # print the spatial subset
#                 spSubset.to_netcdf(file_prefix+basename)
#                 print('downloaded: spatial subset of ' + basename)

#                 # remove the parent
#                 ds.close()
#                 os.remove(basename)
#                 #print('closed')
#                 return(os.path.join(os.getcwd(), file_prefix+basename))

#             else:
#                 ping.close()
#         except:
#             print('File does not exist at this URL: ' + basename)
    

In [ ]:
# def get_x_hourlywrf_PNNL2018(homedir,
#                              spatialbounds,
#                              subdir='PNNL2018/Hourly_WRF_1981_2015/SaukSpatialBounds',
#                              nworkers=4,
#                              start_date='2005-01-01',
#                              end_date='2007-12-31',
#                              time_resolution='H',
#                              time_steps=24,
#                              file_prefix='sp_',
#                              rename_timelatlong_names={'south_north':'SN','west_east':'WE', 'time':'TIME'},
#                              replace_file=True):
#     """
#     get hourly WRF data from a 2018 PNNL WRF run using xarray on netcdf files
#     """
#     # check and generate data directory
#     filedir=os.path.join(homedir, subdir)
#     ogh.ensure_dir(filedir)
    
#     # modify each month between start_date and end_date to year-month
#     dates = [x.strftime('%Y%m%d') for x in pd.date_range(start=start_date, end=end_date, freq='D')]
    
#     # initialize parallel workers
#     da.set_options(pool=ThreadPool(nworkers))
#     ProgressBar().register()
    
#     # generate the list of files to download
#     filelist = compile_x_wrfpnnl2018_raw_locations(dates)
    
#     # download files of interest
#     NetCDFs=[]
#     for url, date in zip(filelist, dates):
#         NetCDFs.append(da.delayed(wget_x_download_spSubset_PNNL)(fileurl=url,
#                                                                  filedate=date,
#                                                                  time_resolution=time_resolution,
#                                                                  time_steps=time_steps,
#                                                                  spatialbounds=spatialbounds,
#                                                                  file_prefix=file_prefix,
#                                                                  rename_timelatlong_names=rename_timelatlong_names,
#                                                                  replace_file=replace_file))
    
#     # run operations
#     outputfiles = da.compute(NetCDFs)[0]
    
#     # reset working directory
#     os.chdir(homedir)
#     return(outputfiles)

In [ ]:
import dask as da
import xarray as xray
from dask.diagnostics import ProgressBar

def netcdf_to_ascii(homedir, subdir, netcdfs, mappingfile, catalog_label, meta_file):
    # initialize list of dataframe outputs
    outfiledict = {}
    
    # generate destination folder
    filedir=os.path.join(homedir, subdir)
    ogh.ensure_dir(filedir)

    # connect with collection of netcdfs
    ds_mf = xray.open_mfdataset(netcdfs, engine = 'netcdf4')

    # convert netcdfs to pandas.Panel API
    ds_pan = ds_mf.to_dataframe().reset_index('TIME')
    
    # generate list of variables
    ds_vars = [ds_var for ds_var in ds_pan.columns
               if ds_var not in ['YEAR','MONTH','DAY','TIME','LAT','LON']]

    # read in gridded cells of interest
    maptable, nstation = ogh.mappingfileToDF(mappingfile, colvar=None)

    # at each latlong of interest
    for ind, eachrow in maptable.iterrows():

        # generate ASCII time-series
        ds_df = ds_pan.loc[eachrow['SN'],eachrow['WE'],:].reset_index(drop=True).loc[:,ds_vars]

        # create file name
        outfilename = os.path.join(filedir,'data_{0}_{1}'.format(eachrow['LAT'],eachrow['LONG_']))

        # save ds_df
        outfiledict[outfilename] = da.delayed(ds_df.to_csv)(path_or_buf=outfilename, sep='\t', header=False, index=False)

    # compute ASCII time-series files
    ProgressBar().register()
    outfiledict = da.compute(outfiledict)[0]
    
    # update metadata file
    meta_file[catalog_label]['variable_info'].update(dict(ds_mf.attrs))
    meta_file[catalog_label]['variable_info'].update(dict(ds_mf.variables))
    meta_file[catalog_label]['variable_list']=np.array(ds_vars)
    
    # catalog the output files
    ogh.addCatalogToMap(outfilepath=mappingfile, maptable=maptable, folderpath=filedir, catalog_label=catalog_label)
    os.chdir(homedir)
    return(list(outfiledict.keys()))

In [ ]:
ds_pan = test2.to_dataframe().reset_index('TIME')
ds_pan

In [ ]:
ds_vars = [ds_var for ds_var in ds_pan.columns
           if ds_var not in ['YEAR','MONTH','DAY','TIME','LAT','LON']]
ds_vars

In [ ]:
pd.date_range(start=meta_file['hourlywrf_pnnl']['start_date'],
              end='2015-12-31 23:00:00',
              freq=meta_file['hourlywrf_pnnl']['temporal_resolution'])

In [ ]:
meta_file['hourlywrf_pnnl']['variable_list']

In [ ]:
# # convert netcdfs to pandas.Panel API
# ds_pan = test2.to_dataframe()[ds_vars].reset_index('TIME')
# print(ds_pan)

# for ind, eachrow in maptable.iterrows():

#         # generate ASCII time-series
#         ds_df = ds_pan.loc[eachrow['SN'],eachrow['WE'],:].reset_index()

In [ ]:
%%time

outputlist = netcdf_to_ascii(homedir=homedir,
                             subdir='PNNL2018/Hourly_WRF_2005_2007/ASCII',
                             netcdfs=outputfiles2,
                             mappingfile=mappingfile1,
                             catalog_label='hourlywrf_pnnl',
                             meta_file=meta_file)

In [ ]:
meta_file['hourlywrf_pnnl']['variable_list']

In [ ]:
maptable, nstation = ogh.mappingfileToDF(mappingfile1,colvar=None)
maptable

In [ ]:
pd.read_table(maptable['hourlywrf_pnnl'][0], header=None)

In [ ]:
ds = xr.open_dataset('PNNL2018/Hourly_WRF_2005_2007/noBC/'+'sp_'+os.path.basename(basename), engine = 'netcdf4')
ds

In [ ]:
# hour = [x.strftime('%Y-%m-%d %H:%M:%S') for x in pd.date_range(start='2005-10-02', periods=24, freq='H')]
# ds['TIME'] = pd.DatetimeIndex(hour)
# ds.TIME

ds

## END TROUBLESHOOTING

## (4) Concatenate netcdf files into one netCDF file and one dataframe with lat,long,time indices : Use xarray function open_mfdataset

In [ ]:
subdir='/PNNL2018/Hourly_WRF_2005_2007/noBC'

os.chdir(homedir+subdir)

In [ ]:
os.getcwd()

In [ ]:
# Download Lat and Lon netcdf file, convert to dataframe
LtLnEl = xr.open_dataset(('data_LatLonGht.nc'), engine = 'netcdf4')
LtLnEl_df = LtLnEl.to_dataframe()
LtLnEl_df.reset_index()

In [ ]:
#NetCDF files(range of dates) to concatenate into a single dataframe
start_date = '2005-10-01'
end_date = '2005-12-30'
dates = [x.strftime('%Y-%m-%d') for x in pd.date_range(start=start_date, end=end_date, freq='D')]
  
#for each day (file) open pnnl netcdf file as xarray dataset, convert all 
#variables to single dataframe, change indexes to corresponding LAT,LON,TIME values, 
#append to dataframe
ds_pan = pd.DataFrame()
ds_pan

In [ ]:
date=dates[0]
#open netcdf file
pnnl = xr.open_dataset('sp_data.' + date +'.nc')
pnnl_df = pnnl.to_dataframe() #index: SN, TIME, WE
pnnl_df = pnnl_df.reset_index(['TIME'])#.sort_values(['SN','WE','TIME'])

pnnl_df

In [ ]:
spatialbounds

In [ ]:
pnnl=pnnl.assign_coords(SN=pnnl.SN, WE=pnnl.WE)
pnnl

In [ ]:
pnnl.sel(SN=slice(4,6), WE=slice(3,5)).to_dataframe()

In [ ]:
pnnl

In [ ]:
pnnl.sel(SN=slice(4,6),WE=slice(4,6)).to_dataframe()

In [ ]:
# Download Lat and Lon netcdf file, convert to dataframe
LtLnEl = xr.open_dataset(('data_LatLonGht.nc'), engine = 'netcdf4')
LtLnEl_df = LtLnEl.to_dataframe()#
LtLnEl_df

In [ ]:
#create series of dates to add to dataset as TIME index
time = pd.date_range(start=date, periods=24, freq='H')
tdf = pd.DataFrame(time, columns = ['TIME'])

tdf

In [ ]:
# Download Lat and Lon netcdf file, convert to dataframe
LtLnEl = xr.open_dataset(('data_LatLonGht.nc'), engine = 'netcdf4')
LtLnEl_df = LtLnEl.to_dataframe()#

#create LAT and LONG index values from spatialbounds
#spatialbounds = {'minx': 49, 'maxx': 57, 'miny': 49, 'maxy': 58} #from notebook usercase 2

#NetCDF files(range of dates) to concatenate into a single dataframe
start_date = '2005-10-01'
end_date = '2005-12-30'
dates = [x.strftime('%Y-%m-%d') for x in pd.date_range(start=start_date, end=end_date, freq='D')]
  
#for each day (file) open pnnl netcdf file as xarray dataset, convert all 
#variables to single dataframe, change indexes to corresponding LAT,LON,TIME values, 
#append to dataframe
ds_pan = pd.DataFrame()
for ini, date in enumerate(dates):
    #for each file

    #open netcdf file
    pnnl = xr.open_dataset('sp_data.' + date +'.nc')
    pnnl_df = pnnl.to_dataframe() #index: SN, TIME, WE
          
    #create series of dates to add to dataset as TIME index
    time = pd.date_range(start=date, periods=24, freq='H')
    tdf = pd.DataFrame(time, columns = ['TIME'])
    


    #(1) set index of pnnl dataframe to indices of lat, long only (remove time indice)
    # =============================================================================
    pnnl_df['index1'] = (pnnl_df.index)
    lst1 = []
    lst2 = []
    lst3 = []
    for ind in list(pnnl_df['index1']):
        lst1.append(ind[0]+ spatialbounds['miny'])#SN index
        lst2.append(ind[2]+ spatialbounds['minx'])#WE index
        lst3.append(ind[1])#TIME index
    pnnl_df['ind1'] = lst1
    pnnl_df['ind2'] = lst2
    pnnl_df['ind3'] = lst3
  
    pnnl_df.set_index(['ind1','ind2'],inplace=True)
    
    #(2) set index of LtLnEl dataframe to indices lat and lon
    # =============================================================================
    
    LtLnEl_df['index2'] = LtLnEl_df.index
     
    lst4 = []
    lst5 = []
    for ind in list(LtLnEl_df['index2']):
        lst4.append(ind[0])#LAT index
        lst5.append(ind[1])#LON index
    # 
    LtLnEl_df['ind1'] = lst4
    LtLnEl_df['ind2'] = lst5
   
    LtLnEl_df.set_index(['ind1','ind2'],inplace=True)

    #(3) Join LAT and LON coordinates to index values to create df3
    # ============================================================================
    
    df3 = pnnl_df.join(LtLnEl_df)
    
    #(4) set index of df3 to time index and join with dataframe containing values for time indices 
    # =============================================================================
     
    df3.set_index(['ind3'],inplace=True)
     
    final = df3.join(tdf)
    
    #(5) Append to final dataframe ds_pan 
    # =============================================================================    
    
    #set index as ind1, ind2 and ind3
    #final.set_index(['LAT','LON','TIME'],inplace=True)
    final.set_index(['LAT','LON'],inplace=True)

    #append final to ds_pan 
    ds_pan = pd.concat([ds_pan, final])
    


In [ ]:
#add lat lon to column data
ds_pan['index3'] = ds_pan.index

lst6 = []
lst7 = []
for ind in list(ds_pan['index3']):
    lst6.append('%.5f'%(ind[0]))#LAT index
    lst7.append('%.5f'%(ind[1]))#LON index
# 
ds_pan['LAT'] = lst6
ds_pan['LON'] = lst7

# ds_pan.set_index(['LAT','LON','TIME'],inplace=True)

In [ ]:
#ds_pan.set_index(['LAT','LON','TIME'],inplace=True)
ds_pan.set_index(['LAT','LON'],inplace=True)

In [ ]:
ds_pan

In [ ]:
#ds_pan.loc['48.12695']
ds_pan.loc['48.126949']

In [ ]:
# read in gridded cells of interest
maptable, nstation = ogh.mappingfileToDF(mappingfile1, colvar=None)

In [ ]:
maptable.head()
val = '%.5f'%(maptable['LAT'][0])
val

In [ ]:
ds_pan.index[0]

In [ ]:
# at each latlong of interest
for ind, eachrow in maptable.iterrows():

    # generate ASCII time-series
    ds_df = ds_pan.loc['%.5f'%(eachrow['LAT']),'%.5f'%(eachrow['LONG_']),:]#.reset_index(drop=True, level=[0,1])

    # create file name
    #outfilename = os.path.join(filedir,'data_{0}_{1}'.format(eachrow['LAT'],eachrow['LONG_']))

    # save ds_df
    #outfiledict[outfilename] = da.delayed(ds_df.to_csv)(path_or_buf=outfilename, sep='\t', header=False, index=False)


# 5. Save results back into HydroShare
<a name="creation"></a>

Using the `hs_utils` library, the results of the Geoprocessing steps above can be saved back into HydroShare.  First, define all of the required metadata for resource creation, i.e. *title*, *abstract*, *keywords*, *content files*.  In addition, we must define the type of resource that will be created, in this case *genericresource*.  

***Note:*** Make sure you save the notebook at this point, so that all notebook changes will be saved into the new HydroShare resource.

### Archive the downloaded data files for collaborative use

Create list of files to save to HydroShare. Verify location and names.

In [ ]:
# %%time
# !tar -zcf livneh2013.tar.gz livneh2013
# !tar -zcf livneh2015.tar.gz livneh2015
# !tar -zcf salathe2014.tar.gz salathe2014

In [ ]:
# # the downloaded tar files
# climate2013_tar = os.path.join(homedir,'livneh2013.tar.gz')
# climate2015_tar = os.path.join(homedir,'livneh2015.tar.gz')
# wrf_tar = os.path.join(homedir,'salathe2014.tar.gz')

In [ ]:
# # for each file downloaded onto the server folder, move to a new HydroShare Generic Resource
# title = 'Downloaded data sets from each study site for each of seven gridded data products'
# abstract = 'This resource contains the downloaded data files for each study site and the gridded cell centroids that intersected within the study area. The file availability is described within the watershed_table file, which summarizes each of the three mapping file catalogs.'
# keywords = ['Sauk', 'Elwha','Rio Salado','climate','hydromet','watershed'] 
# rtype = 'genericresource'

# ##PROBLEM WITH WRITING TO RESOURCE

# # files to migrate ===> NO watershed_table
# #files=[mappingfile1, # sauk
# #       mappingfile2, # elwha
# #       mappingfile3, # riosalado
# #       watershed_table, # watershed summary table
# #       climate2013_tar, # Livneh et al. 2013 raw MET, bc MET, and VIC
# #       climate2015_tar, # Livneh et al. 2015 raw MET, and VIC
# #       wrf_tar] # Salathe et al. 2014 raw WRF and bc WRF

# # files to migrate
# files=[mappingfile1, # sauk
#        mappingfile2, # elwha
#        mappingfile3, # riosalado
#        climate2013_tar, # Livneh et al. 2013 raw MET, bc MET, and VIC
#        climate2015_tar, # Livneh et al. 2015 raw MET, and VIC
#        wrf_tar] # Salathe et al. 2014 raw WRF and bc WRF

# # files to migrate
# #files=[climate2013_tar, # Livneh et al. 2013 raw MET, bc MET, and VIC
# #       climate2015_tar, # Livneh et al. 2015 raw MET, and VIC
# #       wrf_tar] # Salathe et al. 2014 raw WRF and bc WRF


# # create the new resource
# resource_id = hs.createHydroShareResource(title=title, 
#                                           abstract=abstract,
#                                           keywords=keywords, 
#                                           resource_type=rtype, 
#                                           content_files=files, 
#                                           public=False)